# Nielsen Formatter

## Librarys Instalation

In [3]:
import subprocess
import sys

In [4]:
required_packages = ['pandas', 'os', 're', 'glob', 'xlsxwriter', 'datetime']
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

In [5]:
import pandas as pd
import os
import re
import glob
import xlsxwriter
from datetime import datetime

## Data Import

In [ ]:
ROOT_DIR = os.getcwd()
INPUT_PATH = os.path.join(ROOT_DIR, 'input')
OUTPUT_PATH = os.path.join(ROOT_DIR, 'output')
ADD_ID_COLUMN = True # If True id column will be added to the data sheet (reccommended set to True)
SHEET_NUMBER = 3 # Number of sheet to be processed (0, 1, 2, 3, etc.)

In [ ]:
def read_excel_file(input_path):
    excel_folder_path = os.path.join(input_path)
    
    excel_files = glob.glob(os.path.join(excel_folder_path, "*.xlsx"))

    if excel_files:
        first_file = excel_files[0]
        
        df = pd.read_excel(first_file, sheet_name=SHEET_NUMBER, skiprows=8, skipfooter=6)
        return df
    else:
        raise FileNotFoundError(f"Excel file not found in {excel_folder_path}")
    
df = read_excel_file(INPUT_PATH)
df

,Markets,Facts,PRODUCT TYPE,BRAND,SUB-BRAND,PACK TYPE,PACK SIZE IN L,FLAVOUR 2,UPC,Jul 21 - 4 w/e 01/08/21,...,May 24 - 4 w/e 26/05/24,Jun 24 - 5 w/e 30/06/24,Jul 24 - 4 w/e 28/07/24,Aug 24 - 4 w/e 25/08/24,Latest 52 Wks 2YA - w/e 28/08/22,Latest 52 Wks YA - w/e 27/08/23,Latest 52 Wks - w/e 25/08/24,YTD 2YA - 34 w/e 28/08/22,YTD YA - 34 w/e 27/08/23,YTD - 34 w/e 25/08/24
0,Total Poland,Sales Value,ENERGY DRINK,DZIK (WK (BEVERAGES)),DZIK ZERO CALORIE,TIN,0.25,WITHOUT EXTRA FLAVOUR,5902176738907,0.00000,...,8.035208e+03,1.792181e+04,2.182622e+05,2.941679e+05,785611.741507,3.705364e+05,5.493111e+05,785611.741507,5.732828e+03,5.481349e+05
1,Total Poland,Sales Value,ENERGY DRINK,DZIK (WK (BEVERAGES)),DZIK ZERO CALORIE,TIN,0.5,CHERRY,5904988309740,0.00000,...,1.771199e+06,2.023930e+06,1.433597e+06,1.396119e+06,0.000000,5.976480e+06,1.893973e+07,0.000000,5.976480e+06,1.246409e+07
2,Total Poland,Sales Value,ENERGY DRINK,DZIK (WK (BEVERAGES)),DZIK ZERO CALORIE,TIN,0.5,GRAPE,5904988309672,0.00000,...,8.175001e+05,3.738838e+05,2.622323e+05,8.330510e+05,0.000000,6.622683e+06,1.011446e+07,0.000000,6.622683e+06,5.004013e+06
3,Total Poland,Sales Value,ENERGY DRINK,DZIK (WK (BEVERAGES)),DZIK ZERO CALORIE,TIN,0.5,GRAPE,5904988309689,0.00000,...,4.445902e+03,1.091603e+03,9.775068e+02,3.155200e+03,0.000000,9.019938e+04,5.316310e+04,0.000000,9.019938e+04,2.523673e+04
4,Total Poland,Sales Value,ENERGY DRINK,DZIK (WK (BEVERAGES)),DZIK ZERO CALORIE,TIN,0.5,LEMON / CITRON,5904988310036,0.00000,...,2.203728e+04,5.508814e+04,3.691336e+04,5.032592e+04,0.000000,0.000000e+00,1.913331e+05,0.000000,0.000000e+00,1.913331e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54295,Kanały alternatywne,Total Weighted Distribution Points (TDP),ENERGY DRINK,BLACK (FOODCARE),BLACK ZERO,TIN,0.25,WITHOUT EXTRA FLAVOUR,5900552022473,0.00000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,4.976940e-04,0.000000,0.000000e+00,0.000000e+00
54296,Kanały alternatywne,Total Weighted Distribution Points (TDP),ENERGY DRINK,BLACK (FOODCARE),BLACK ZERO,TIN,0.25,WITHOUT EXTRA FLAVOUR,5900552027416SKU,29.02253,...,2.986164e-03,9.953881e-04,0.000000e+00,0.000000e+00,30.828661,3.509589e+01,2.087080e+01,32.622848,3.592405e+01,1.424201e+01
54297,Kanały alternatywne,Total Weighted Distribution Points (TDP),ENERGY DRINK,BLACK (FOODCARE),BLACK ZERO,TIN,0.33,WITHOUT EXTRA FLAVOUR,5900552056171,0.00000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.078138,1.045157e-02,0.000000e+00,0.039816,0.000000e+00,0.000000e+00
54298,Kanały alternatywne,Total Weighted Distribution Points (TDP),ENERGY DRINK,BLACK (FOODCARE),BLACK ZERO,TIN,0.5,CANTALOUPE,5900552087656,0.00000,...,2.349116e-01,3.812336e-01,1.298981e-01,0.000000e+00,0.000000,0.000000e+00,7.863566e-02,0.000000,0.000000e+00,1.149673e-01


## Changing Date Format

### Extracting date columns

In [303]:
columns = df.columns

date_pattern = re.compile(r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\b\s\d{2}.*\d{2}/\d{2}/\d{2}')

try:
  date_columns = [col for col in columns if date_pattern.search(col)]
except Exception as e:
  print(f"Date format has already been formatted: Error: {e}")
else:
  df_date_columns = df[date_columns]


In [304]:
unique_date_columns = df.columns[-6:]
new_unique_date_columns = df.columns[-6:].str.replace(" - w/e", "", regex=False)
new_unique_date_columns = new_unique_date_columns.str.replace(" w/e", "", regex=False)
new_unique_date_columns

Index(['Latest 52 Wks 2YA 28/08/22', 'Latest 52 Wks YA 27/08/23',
       'Latest 52 Wks 25/08/24', 'YTD 2YA - 34 28/08/22',
       'YTD YA - 34 27/08/23', 'YTD - 34 25/08/24'],
      dtype='object')

### Renaming date columns

In [305]:
new_date_columns = [str(datetime.strptime(date.split(" - ")[0], '%b %y').date())[:7] for date in df_date_columns]
df.rename(columns={date: new_date for date, new_date in zip(date_columns, new_date_columns)}, inplace=True)

new_unique_date_columns_formatted = []
latests = [
    'Latest 52 Wks 2YA ',
    'Latest 52 Wks YA ',
    'Latest 52 Wks ',
 ]

for date in new_unique_date_columns:
    part1 = str(date[:-8])
    part2 = str(datetime.strptime(date[-5:], '%m/%y').strftime('%Y-%m'))

    if part1 == latests[0]:
        part1 = 'MAT 2LY '
    elif part1 == latests[1]:
        part1 = 'MAT LY '
    elif part1 == latests[-1]:
        part1 = 'MAT TY '
    new_unique_date_columns_formatted.append(part1 + part2)

new_unique_date_columns_formatted

df.rename(columns={date: unique_date for date, unique_date in zip(unique_date_columns, new_unique_date_columns_formatted)}, inplace=True)


In [306]:
df.columns

Index(['Markets', 'Facts', 'PRODUCT TYPE', 'BRAND', 'SUB-BRAND', 'PACK TYPE',
       'PACK SIZE IN L', 'FLAVOUR 2', 'UPC', '2021-07', '2021-08', '2021-09',
       '2021-10', '2021-11', '2021-12', '2022-01', '2022-02', '2022-03',
       '2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09',
       '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
       '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09',
       '2023-10', '2023-11', '2023-12', '2024-01', '2024-02', '2024-03',
       '2024-04', '2024-05', '2024-06', '2024-07', '2024-08',
       'MAT 2LY 2022-08', 'MAT LY 2023-08', 'MAT TY 2024-08',
       'YTD 2YA - 34 2022-08', 'YTD YA - 34 2023-08', 'YTD - 34 2024-08'],
      dtype='object')

## Filtering

### Filtering Years

In [307]:
def years_filter(df):
    years_to_delete = [col for col in df.columns[:-6] if re.search(r'\d{4}-\d{2}', col)]
    years_to_delete = years_to_delete[:-24]

    last_to_delete = years_to_delete[-1]
    unique_years_to_delete =  [col for col in df.columns if col in [f'MAT 2LY {last_to_delete}', f'YTD 2YA - 34 {last_to_delete}']]
    
    to_delete = years_to_delete + unique_years_to_delete

    return to_delete
  
years_to_delete = years_filter(df)
df.drop(columns=years_to_delete, inplace=True)

In [308]:
df.columns

Index(['Markets', 'Facts', 'PRODUCT TYPE', 'BRAND', 'SUB-BRAND', 'PACK TYPE',
       'PACK SIZE IN L', 'FLAVOUR 2', 'UPC', '2022-09', '2022-10', '2022-11',
       '2022-12', '2023-01', '2023-02', '2023-03', '2023-04', '2023-05',
       '2023-06', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11',
       '2023-12', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05',
       '2024-06', '2024-07', '2024-08', 'MAT LY 2023-08', 'MAT TY 2024-08',
       'YTD YA - 34 2023-08', 'YTD - 34 2024-08'],
      dtype='object')

### Filtering Facts

In [309]:
facts = df.groupby('Facts')
facts.describe()

2022-09                 \
                                           count           mean   
Facts                                                             
Avg No. of Items per Store                4525.0       0.413511   
Numeric Distribution                      4525.0       1.044337   
Price per Sales EQ                        4525.0       0.647168   
Price per Sales Unit                      4525.0       2.452999   
ROS Units                                 4525.0       0.750726   
Sales EQ                                  4525.0   13304.904068   
Sales Units                               4525.0   25443.244615   
Sales Value                               4525.0  136947.765208   
TDP Numeric Distribution                  4525.0       5.043018   
Total Weighted Distribution Points (TDP)  4525.0       3.868262   
Unwghtd ROS Units                         4525.0       1.223504   
Weighted Distribution                     4525.0       1.431459   

                                                                        \
                                                    std  min  25%  50%   
Facts                                                                    
Avg No. of Items per Store                     0.444605  0.0  0.0  0.0   
Numeric Distribution                           2.784881  0.0  0.0  0.0   
Price per Sales EQ                             0.902621  0.0  0.0  0.0   
Price per Sales Unit                           7.372295  0.0  0.0  0.0   
ROS Units                                      2.322737  0.0  0.0  0.0   
Sales EQ                                   85430.266305  0.0  0.0  0.0   
Sales Units                               205509.863208  0.0  0.0  0.0   
Sales Value                               945199.061767  0.0  0.0  0.0   
TDP Numeric Distribution                      13.447962  0.0  0.0  0.0   
Total Weighted Distribution Points (TDP)       9.404390  0.0  0.0  0.0   
Unwghtd ROS Units                              3.400213  0.0  0.0  0.0   
Weighted Distribution                          3.480116  0.0  0.0  0.0   

                                                                    2022-10  \
                                                  75%           max   count   
Facts                                                                         
Avg No. of Items per Store                   0.891442  8.923333e-01  4525.0   
Numeric Distribution                         0.147789  1.766257e+01  4525.0   
Price per Sales EQ                           1.200483  7.471755e+00  4525.0   
Price per Sales Unit                         2.307561  9.112342e+01  4525.0   
ROS Units                                    0.692966  6.701252e+01  4525.0   
Sales EQ                                   887.527025  2.592633e+06  4525.0   
Sales Units                                984.235726  6.639884e+06  4525.0   
Sales Value                               7668.782037  2.971799e+07  4525.0   
TDP Numeric Distribution                     0.713659  8.529109e+01  4525.0   
Total Weighted Distribution Points (TDP)     0.625104  4.963055e+01  4525.0   
Unwghtd ROS Units                            1.393713  9.690643e+01  4525.0   
Weighted Distribution                        0.231321  1.836589e+01  4525.0   

                                                         ...  \
                                                   mean  ...   
Facts                                                    ...   
Avg No. of Items per Store                     0.416266  ...   
Numeric Distribution                           1.060704  ...   
Price per Sales EQ                             0.652873  ...   
Price per Sales Unit                           2.406372  ...   
ROS Units                                      0.757234  ...   
Sales EQ                                   10560.162729  ...   
Sales Units                                20389.712186  ...   
Sales Value                               109655.714462  ...   
TDP Numeric Distribution          

### Filtering UPC

In [310]:
def check_last_3(upc):
    return upc[-3:] in ['BP1', 'BP2', 'BP3', 'BP4', 'BP5', 'BP6', 'BP7', 'BP8', 'BP9']

def check_last_4(upc):
    return upc[-4:] in ['BP10', 'BP11', 'BP12']

df = df[~df['UPC'].apply(check_last_3)]
df = df[~df['UPC'].apply(check_last_4)]

In [311]:
def rename_sku(upc):
    if upc[-3:] == 'SKU':
        return upc[:-3]
    else:
        return upc

df['UPC'] = df['UPC'].apply(rename_sku)

## Adding TOTAL MW and TOTAL Rows

In [312]:
facts_filter = [
    # 'Avg No. of Items per Store',
    'Numeric Distribution',
    # 'Price per Sales EQ',
    # 'Price per Sales Unit',
    # 'ROS Units',
    # 'Sales EQ',
    'Sales Units',
    'Sales Value',
    # 'TDP Numeric Distribution',
    # 'Total Weighted Distribution Points (TDP)',
    # 'Unwghtd ROS Units',
    'Weighted Distribution'
]

aggregation_rules = {
    'Avg No. of Items per Store': 'mean',
    'Numeric Distribution': 'mean',
    'Price per Sales EQ': 'mean',
    'Price per Sales Unit': 'mean',
    'ROS Units': 'mean',
    'Sales EQ': 'sum',
    'Sales Units': 'sum',
    'Sales Value': 'sum',
    'TDP Numeric Distribution': 'sum',
    'Total Weighted Distribution Points (TDP)': 'sum',
    'Unwghtd ROS Units': 'mean',
    'Weighted Distribution': 'mean'
}


In [313]:
filtered_df = df[df['Facts'].isin(facts_filter)].reset_index(drop=True)

df = filtered_df
df['Facts'].unique()

array(['Sales Value', 'Sales Units', 'Numeric Distribution',
       'Weighted Distribution'], dtype=object)

## Adding ID

In [314]:
id_columns = ['Markets', 'Facts', 'BRAND', 'FLAVOUR 2', "UPC"]

def add_id(df, id_columns):
    """
    Adds an 'id1' column to the DataFrame by concatenating the specified columns.
    """
    missing_columns = [col for col in id_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Missing columns: {missing_columns}")

    df['id1'] = df[id_columns].astype(str).agg(''.join, axis=1)

if ADD_ID_COLUMN:
    add_id(df, id_columns)


## XLSX Writer

In [315]:
SHEET_NAMES = {
    "DATA": "dane",
    "LISTS": "listy",
    "LINE": "linia"
}
FORMULAS_LENGTH = 50
MERGE_RANGE = 3
CALCULATION_NAMES = [
    "Sales Units",
    "Sales Value",
    "Average Price per Unit",
    "Share in Units Sold",
    "Value Share",
    "Numeric Distribution TOTAL",
    "Weighted Value Distribution",
    "UNIT INDEX",
    "VALUE INDEX"
]
CALCULATION_LEGEND = {
    "Sales Units": "dynamics",
    "Sales Value": "dynamics",
    "Average Price per Unit": "dynamics",
    "Share in Units Sold": "change",
    "Value Share": "change",
    "Numeric Distribution TOTAL": "change",
    "Weighted Value Distribution": "change",
    "UNIT INDEX": "dynamics",
    "VALUE INDEX": "dynamics"
}

In [316]:
writer = pd.ExcelWriter('output/output.xlsx', engine='xlsxwriter')

### Converting to Excel

In [317]:
df.to_excel(writer, sheet_name=SHEET_NAMES["DATA"], index=False)

In [318]:
workbook  = writer.book
data_ws = writer.sheets[SHEET_NAMES["DATA"]]
lists_ws = workbook.add_worksheet(SHEET_NAMES["LISTS"])
line_ws = workbook.add_worksheet(SHEET_NAMES["LINE"])

### Adding lists

In [319]:
def lists_add(df, lists_ws, months_to_format):
    """
    This function creates mandatory lists in the provided worksheet:
    - Markets: Unique market names from the DataFrame.
    - Facts: Unique fact names from the DataFrame.
    - Months: A list of months (1-12), LY (last year), and TY (this year) data.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the data.
    - lists_ws (xlsxwriter.worksheet.Worksheet): The worksheet where the lists will be written.
    - months_to_format (list): A list containing month-related data to be formatted.

    Returns:
    - tuple: A tuple containing unique markets and unique facts as numpy arrays.
    """
    unique_markets = df['Markets'].unique()
    for row_num, data in enumerate(unique_markets):
        lists_ws.write(row_num, 0, data)

    unique_facts = df['Facts'].unique()
    for row_num, data in enumerate(unique_facts):
        lists_ws.write(row_num, 1, data)

    for col_idx, column in enumerate(months_to_format):
        for row_idx, row in enumerate(column):
            if isinstance(row, str) and row.startswith('='):
                lists_ws.write_formula(row_idx, col_idx + 2, row)
            else:
                lists_ws.write(row_idx, col_idx + 2, row)

    return unique_markets, unique_facts

date_columns = [
    str(datetime.strptime(col, '%Y-%m').date().strftime(f'%b %y'))
    for col in df.columns if re.match(r'\d{4}-\d{2}', col)
]

months_to_format = [
    list(range(1, 13)),  # Numbers 1 to 12

    # LY (last year)
    [date_columns[0]] +
    [f'{date_columns[0]}-{date_columns[i]}' for i in range(1, 11)] +
    [f'=XLOOKUP("MAT LY", {SHEET_NAMES["DATA"]}!$1:$1,{SHEET_NAMES["DATA"]}!$1:$1, "MAT LY", 1)'],

    # TY (this year)
    [date_columns[0 + 12]] +
    [f'{date_columns[12]}-{date_columns[i + 12]}' for i in range(1, 11)] +
    [f'=XLOOKUP("MAT TY", {SHEET_NAMES["DATA"]}!$1:$1,{SHEET_NAMES["DATA"]}!$1:$1, "MAT TY", 1)']
]

lists = lists_add(df, lists_ws, months_to_format)

markets = lists[0]
facts = lists[1]

In [320]:
months_to_format

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 ['Sep 22',
  'Sep 22-Oct 22',
  'Sep 22-Nov 22',
  'Sep 22-Dec 22',
  'Sep 22-Jan 23',
  'Sep 22-Feb 23',
  'Sep 22-Mar 23',
  'Sep 22-Apr 23',
  'Sep 22-May 23',
  'Sep 22-Jun 23',
  'Sep 22-Jul 23',
  '=XLOOKUP("MAT LY", dane!$1:$1,dane!$1:$1, "MAT LY", 1)'],
 ['Sep 23',
  'Sep 23-Oct 23',
  'Sep 23-Nov 23',
  'Sep 23-Dec 23',
  'Sep 23-Jan 24',
  'Sep 23-Feb 24',
  'Sep 23-Mar 24',
  'Sep 23-Apr 24',
  'Sep 23-May 24',
  'Sep 23-Jun 24',
  'Sep 23-Jul 24',
  '=XLOOKUP("MAT TY", dane!$1:$1,dane!$1:$1, "MAT TY", 1)']]

### Formatting Sheets

#### Defining Colurs

In [321]:
bg_light_green = workbook.add_format({'bg_color': '#EBF1DE'})
bg_yellow = workbook.add_format({'bg_color': '#FCFF93'})
bg_gray = workbook.add_format({'bg_color': '#C0C0C0'})
bg_light_gray = workbook.add_format({'bg_color': '#D9D9D9'})

bg_light_olive_green = workbook.add_format({'bg_color': '#D8E4BC'})
bg_olive_green = workbook.add_format({'bg_color': '#C4D79B'})
bg_mint = workbook.add_format({'bg_color': '#C7EBC8'})
bg_light_aquamarine = workbook.add_format({'bg_color': '#DAEEF3'})
bg_aquamarine = workbook.add_format({'bg_color': '#B7DEE8'})
bg_light_purple = workbook.add_format({'bg_color': '#CCC0DA'})
bg_purple = workbook.add_format({'bg_color': '#B1A0C7'})
bg_light_red = workbook.add_format({'bg_color': '#F2DCDB'})
bg_red = workbook.add_format({'bg_color': '#E6B8B7'})

col_dark_aquamarine = workbook.add_format({'font_color': '#31869B'})
col_green = workbook.add_format({'font_color': '#00B050'})
col_beige = workbook.add_format({'font_color': '#948A54'})
col_red = workbook.add_format({'font_color': '#FF0000'})

bg_for_calculations = [bg_light_olive_green, bg_olive_green, bg_mint, bg_light_aquamarine, bg_aquamarine, bg_light_purple, bg_purple, bg_light_red, bg_red]

#### Main Menu Styling And Adding Lists Functionality

In [322]:
format = workbook.add_format({
    'bg_color': bg_red.bg_color,
    'align': 'center',
    'valign': 'vcenter',
    'font_size': 12,
    'bold': True,
})

line_ws.write(0, 0, markets[0], format)
line_ws.data_validation('A1', {
    'validate': 'list',
    'source': f'={SHEET_NAMES["LISTS"]}!$A$1:$A${len(markets)}',
})
line_ws.write_comment(0, 0, "Use This Cell to Select Market", {'x_scale': 1.5, 'y_scale': 0.5})

format = workbook.add_format({
    'bg_color': bg_aquamarine.bg_color,
    'align': 'center',
    'valign': 'vcenter',
    'font_size': 12,
    'bold': True,
})

line_ws.write(0, 1, 12, format)
line_ws.data_validation('B1', {
    'validate': 'list',
    'source': f'={SHEET_NAMES["LISTS"]}!$C$1:$C${len(months_to_format[0])}',
})
line_ws.write_comment(0, 1, "Use This Cell to Select Months Period", {'x_scale': 1.5, 'y_scale': 0.5})

combined_format = workbook.add_format({
    'bg_color': bg_yellow.bg_color,
    'align': 'center',
    'valign': 'vcenter',
    'font_size': 12,
    'bold': True,
})
line_ws.write(1, 0, '', combined_format)
line_ws.write(1, 1, 'UPC', combined_format)

0

#### Parameters Section

In [323]:
parameters = ['Product', 'Brand', 'Pack Type', 'PACK SIZE IN L', 'Flavour 2']
for idx, parameter in enumerate(parameters):
    line_ws.write(0, 2 + idx, parameter, bg_light_green)
    combined_format = workbook.add_format({
        'bg_color': bg_light_green.bg_color,
        'align': 'center',
        'valign': 'vcenter'
    })
    line_ws.merge_range(0, 2 + idx, 1, 2 + idx, parameter, combined_format)

#### Calculations Section

In [324]:
for idx, parameter in enumerate(CALCULATION_NAMES):
    col_idx = 2 + len(parameters) + idx * MERGE_RANGE
    bg_color = bg_for_calculations[idx % len(bg_for_calculations)]
    line_ws.write(0, col_idx, parameter, bg_color)
    combined_format = workbook.add_format({'bg_color': bg_color.bg_color, 'align': 'center', 'valign': 'vcenter'})
    line_ws.merge_range(0, col_idx, 0, col_idx + 2, parameter, combined_format)

In [325]:
xlooukup_d = f"=XLOOKUP({SHEET_NAMES['LINE']}!$B$1, {SHEET_NAMES['LISTS']}!$C:$C, {SHEET_NAMES['LISTS']}!$D:$D)"
xlooukup_e = f"=XLOOKUP({SHEET_NAMES['LINE']}!$B$1, {SHEET_NAMES['LISTS']}!$C:$C, {SHEET_NAMES['LISTS']}!$E:$E)"

calculations = []
for name in CALCULATION_LEGEND:
    calculations.append(xlooukup_d)
    calculations.append(xlooukup_e)
    calculations.append(CALCULATION_LEGEND[name])

In [326]:
color_idx = 0
for idx, parameter in enumerate(calculations):
    if idx % MERGE_RANGE == 0 and idx != 0:
        color_idx += 1
        if color_idx >= len(bg_for_calculations):
            color_idx = 0
    
    combined_format = workbook.add_format({
        'bg_color': bg_for_calculations[color_idx].bg_color,
        'align': 'center',
        'valign': 'vcenter',
        'text_wrap': True
    })

    line_ws.write(1, 2 + len(parameters) + idx, parameter, combined_format)

#### Facts Section

In [327]:
facts_colors = [col_dark_aquamarine, col_green, col_beige, col_red]
for idx, parameter in enumerate(facts):
    combined_format = workbook.add_format({
        'font_color': facts_colors[idx].font_color,
        'align': 'center',
        'valign': 'vcenter'
    })
    facts_colors_row = 2 + len(parameters) + len(CALCULATION_NAMES) * MERGE_RANGE + idx
    line_ws.write(0, facts_colors_row, parameter, combined_format)
    line_ws.merge_range(0, facts_colors_row, 1, facts_colors_row, parameter, combined_format)

In [328]:
multiple_dates_by_facts = date_columns * len(facts)
color_idx = 0
for idx, parameter in enumerate(multiple_dates_by_facts):
    if idx % len(date_columns) == 0 and idx != 0:
        color_idx += 1
        if color_idx >= len(facts_colors):
            color_idx = 0
    combined_format = workbook.add_format({
        'font_color': facts_colors[color_idx].font_color,
        'align': 'center',
        'valign': 'vcenter'
    })
    multiple_dates_by_facts_row = 2 + len(parameters) + len(CALCULATION_NAMES) * MERGE_RANGE + len(facts) + idx
    line_ws.write(0, multiple_dates_by_facts_row, parameter, combined_format)
    line_ws.merge_range(0, multiple_dates_by_facts_row, 1, multiple_dates_by_facts_row, parameter, combined_format)

#### Enchancing Menu According to Row Length

In [329]:
first_row_length = 2 + len(parameters) + len(CALCULATION_NAMES) * MERGE_RANGE + len(facts) + len(multiple_dates_by_facts)

bg_format = workbook.add_format({
    'bg_color': bg_gray.bg_color,
})
combined_format = workbook.add_format({
    'bg_color': bg_gray.bg_color,
    'valign': 'vcenter',
    'font_size': 12,
})

for idx in range(first_row_length):
    line_ws.write(2, idx, '', bg_format)
line_ws.write(2, 1, 'TOTAL', combined_format)

bg_format = workbook.add_format({
    'bg_color': bg_light_gray.bg_color,
})
combined_format = workbook.add_format({
    'bg_color': bg_light_gray.bg_color,
    'valign': 'vcenter',
    'font_size': 12,
})

for idx in range(first_row_length):
    line_ws.write(3, idx, '', bg_format)
line_ws.write(3, 1, 'TOTAL MW', combined_format)

0

### Formatter Formulas

#### Changing Numbers to Letters According to Excel Columns Functionality

In [330]:
import string
letters = string.ascii_uppercase

facts_python_format = [fact for fact in facts]
python_columns = [col.upper() for col in df.columns.tolist()]

upc_idx = python_columns.index('UPC')
upc_column = letters[upc_idx]

total = markets[0]
first_row_length = 2 + len(parameters) + len(CALCULATION_NAMES) * MERGE_RANGE + len(facts)

alphabet = string.ascii_uppercase
alphabet_letters = [f"{col}" for col in alphabet]
letters = alphabet_letters + [first_letter + second_letter for first_letter in alphabet for second_letter in alphabet]

#### Defining Parameter Columns Location

In [331]:
for col, parameter in enumerate(parameters):
    try:
        parameter_idx = python_columns.index(parameter.upper())
    except ValueError:
        continue
    else:
        correct_column = letters[parameter_idx]
        print(f"{parameter}: {correct_column}")
        for idx in range(FORMULAS_LENGTH):
            product_info_function = f'=IF(XLOOKUP({SHEET_NAMES["LINE"]}!$B{idx + 5},{SHEET_NAMES["DATA"]}!${upc_column}:${upc_column},{SHEET_NAMES["DATA"]}!{correct_column}:{correct_column},0)=0,"",XLOOKUP({SHEET_NAMES["LINE"]}!$B{idx + 5},{SHEET_NAMES["DATA"]}!${upc_column}:${upc_column},{SHEET_NAMES["DATA"]}!{correct_column}:{correct_column},""))'
            line_ws.write(idx + 4, col + 2, product_info_function)

Brand: D
Pack Type: F
PACK SIZE IN L: G
Flavour 2: H


In [332]:
def concat_format(workbook, color, type):
    """
    Creates and returns an Excel format object with specific formatting options.

    Args:
        workbook (xlsxwriter.Workbook): The workbook object to which the format will be applied.
        color (str): The background color to apply. Supported values are:
            - "gray": Applies a gray background color.
            - "light_gray": Applies a light gray background color.
            - Any other value will result in no background color.
        type (str): The type of formatting to apply. Supported values are:
            - "numeric": Applies a numeric format with French locale.
            - "percent": Applies a percentage format.
            - "average": Applies a format with two decimal places.
            - Any other value will result in no specific formatting.

    Returns:
        xlsxwriter.format.Format: The created format object with the specified options, or None if no valid type is provided.
    """
    if color == "gray":
        bg_color = '#C0C0C0'
    elif color == "light_gray":
        bg_color = '#D9D9D9'
    else:
        bg_color = False

    if type == "numeric":
        format = workbook.add_format({
            'num_format': '[$-fr-FR]#,##0',
            'bg_color': bg_color,
    })
    elif type == "percent":
        format = workbook.add_format({
            'num_format': '0.0%',
            'bg_color': bg_color,
        })
    elif type == "average":
        format = workbook.add_format({
            'num_format': '0.00',
            'bg_color': bg_color,
        })
    else:
        format = None

    return format

#### Facts Calculations

In [333]:
for fact_idx, fact in enumerate(facts_python_format):
    for date_idx, date in enumerate(date_columns):
        date_column = letters[upc_idx + 1 + date_idx]
        
        if aggregation_rules[fact] == 'sum':
            excel_function = "SUMIFS"
            format = "numeric"
        else:
            excel_function = "AVERAGEIFS"
            format = "average"

        correct_column = len(date_columns) * (fact_idx) + first_row_length + date_idx

        gray_combined_format = concat_format(workbook, color="gray", type=format)
        total_mw = f'={excel_function}({SHEET_NAMES["DATA"]}!{date_column}:{date_column},{SHEET_NAMES["DATA"]}!$A:$A,"{total}",{SHEET_NAMES["DATA"]}!$B:$B,"{fact}")'
        line_ws.write(2, correct_column, total_mw, gray_combined_format)

        light_gray_combined_format = concat_format(workbook, color="light_gray", type=format)
        total_single_market = f'={excel_function}({SHEET_NAMES["DATA"]}!{date_column}:{date_column},{SHEET_NAMES["DATA"]}!$A:$A,$A$1,{SHEET_NAMES["DATA"]}!$B:$B,"{fact}")'
        line_ws.write(3, correct_column, total_single_market, light_gray_combined_format)

        normal_format = concat_format(workbook, color="", type=format)
        for idx in range(FORMULAS_LENGTH):
            id1_column = letters[python_columns.index("ID1")]
            correct_idx = idx + 5
            value_finder = f'=IFERROR(XLOOKUP($A$1&"{fact}"&$D{correct_idx}&$G{correct_idx}&$B{correct_idx}, {SHEET_NAMES["DATA"]}!${id1_column}:${id1_column}, {SHEET_NAMES["DATA"]}!{date_column}:{date_column}), "")'
            line_ws.write(idx + 4, correct_column, value_finder, normal_format)


#### Defining Positions of the Calculation Names

In [334]:
facts_preparation = {value: first_row_length + len(date_columns) * (key + 1) for key, value in enumerate(facts_python_format)}
facts_preparation

{'Sales Value': 62,
 'Sales Units': 86,
 'Numeric Distribution': 110,
 'Weighted Distribution': 134}

In [335]:
fact_names_change = {
    'Sales Units': 'Sales Units',
    'Sales Value': 'Sales Value',
    'Numeric Distribution': 'Numeric Distribution TOTAL',
    'Weighted Distribution': 'Weighted Value Distribution'
}

facts_ready_to_prepare = {fact_names_change[fact]: value for fact, value in facts_preparation.items()}
facts_ready_to_prepare

{'Sales Value': 62,
 'Sales Units': 86,
 'Numeric Distribution TOTAL': 110,
 'Weighted Value Distribution': 134}

### Calculations Section

In [336]:
def prepare_facts(column_num):
    """
    Prepares the Excel column letters for the given fact's last year (LY) and this year (TY) data.

    This function calculates the column letters corresponding to the last year (LY) and this year (TY) 
    data for a specific fact based on the provided column number. It assumes that the columns are 
    organized sequentially with a fixed number of months in a year.

    Args:
        column_num (int): The column number corresponding to the fact's TY data in the Excel sheet.

    Returns:
        tuple: A tuple containing two strings:
            - fact_ly (str): The column letter for the fact's last year (LY) data.
            - fact_ty (str): The column letter for the fact's this year (TY) data.
    """
    months_in_year = 12

    fact_ly = letters[column_num - 1 - months_in_year]
    fact_ty = letters[column_num - 1]

    return fact_ly, fact_ty

In [337]:
def prepare_calculation(name, row, start_column, merge_range, letters, **kwargs):
    """
    Prepares Excel formulas for specific calculations based on the given name, row, and column positions.

    Args:
        name (str): The name of the calculation to prepare. Supported names include:
            - "Average Price per Unit"
            - "Share in Units Sold"
            - "Value Share"
            - "Numeric Distribution TOTAL"
            - "Weighted Value Distribution"
            - "UNIT INDEX"
            - "VALUE INDEX"
        row (int): The row number in the Excel sheet where the calculation will be applied.
        start_column (int): The starting column index for the calculation.
        merge_range (int): The range of columns to merge for each calculation.
        letters (list): A list of Excel column letters corresponding to column indices.
        **kwargs: Additional keyword arguments, including:
            - fact_ly (str): The column letter for the "last year" fact.
            - fact_ty (str): The column letter for the "this year" fact.
            - prepared_facts (dict): A dictionary of pre-calculated fact positions for "UNIT INDEX" and "VALUE INDEX".

    Returns:
        tuple: A tuple containing two strings:
            - calculation_ly: The Excel formula for the "last year" calculation.
            - calculation_ty: The Excel formula for the "this year" calculation.

    Raises:
        ValueError: If an unsupported calculation name is provided.

    Notes:
        - The function dynamically generates Excel formulas for various calculations, including averages,
          shares, and indices, based on the provided parameters.
    """
    if name == "Average Price per Unit":
        sales_value_location_ly = letters[start_column - merge_range]
        sales_units_location_ly = letters[start_column - 2 * merge_range]
        sales_value_location_ty = letters[start_column - merge_range + 1]
        sales_units_location_ty = letters[start_column - 2 * merge_range + 1]
        calculation_ly = f'=IFERROR({sales_value_location_ly}{row}/{sales_units_location_ly}{row}, "")'
        calculation_ty = f'=IFERROR({sales_value_location_ty}{row}/{sales_units_location_ty}{row}, "")'
    elif name == "Share in Units Sold":
        sales_units_location_ly = letters[start_column - 3 * merge_range]
        sales_units_location_ty = letters[start_column - 3 * merge_range + 1]
        calculation_ly = f'=IFERROR({sales_units_location_ly}{row}/MAX({sales_units_location_ly}${row}:{sales_units_location_ly}${merge_range}), "")'
        calculation_ty = f'=IFERROR({sales_units_location_ty}{row}/MAX({sales_units_location_ty}${row}:{sales_units_location_ty}${merge_range}), "")'
    elif name == "Value Share":
        sales_value_location_ly = letters[start_column - 3 * merge_range]
        sales_value_location_ty = letters[start_column - 3 * merge_range + 1]
        calculation_ly = f'=IFERROR({sales_value_location_ly}{row}/MAX({sales_value_location_ly}${row}:{sales_value_location_ly}${merge_range}), "")'
        calculation_ty = f'=IFERROR({sales_value_location_ty}{row}/MAX({sales_value_location_ty}${row}:{sales_value_location_ty}${merge_range}), "")'
    elif name == "Numeric Distribution TOTAL" or name == "Weighted Value Distribution":
        fact_ly = kwargs.get('fact_ly')
        fact_ty = kwargs.get('fact_ty')
        calculation_ly = f'=IFERROR(AVERAGE(OFFSET({fact_ly}{row},0,-($B$1-1)):{fact_ly}{row})/100,"")'
        calculation_ty = f'=IFERROR(AVERAGE(OFFSET({fact_ty}{row},0,-($B$1-1)):{fact_ty}{row})/100,"")'
    elif name == "UNIT INDEX" or name == "VALUE INDEX":
        prepared_facts = kwargs.get('prepared_facts')
        fact_ly = 0
        fact_ty = 1
        if name == "UNIT INDEX":
            sales_loc_ly = prepared_facts["Sales Units"][fact_ly]
            sales_loc_ty = prepared_facts["Sales Units"][fact_ty]
        else:
            sales_loc_ly = prepared_facts["Sales Value"][fact_ly]
            sales_loc_ty = prepared_facts["Sales Value"][fact_ty]
        weighted_dist_loc_ly = prepared_facts["Weighted Value Distribution"][fact_ly]
        weighted_dist_loc_ty = prepared_facts["Weighted Value Distribution"][fact_ty]
        calculation_ly = f'=IFERROR(INDEX(LINEST(OFFSET({sales_loc_ly}{row},0,-($B$1-1)):{sales_loc_ly}{row}, OFFSET({weighted_dist_loc_ly}{row},0,-($B$1-1)):{weighted_dist_loc_ly}{row}, 0, 0), 1, 1), "")'
        calculation_ty = f'=IFERROR(INDEX(LINEST(OFFSET({sales_loc_ty}{row},0,-($B$1-1)):{sales_loc_ty}{row}, OFFSET({weighted_dist_loc_ty}{row},0,-($B$1-1)):{weighted_dist_loc_ty}{row}, 0, 0), 1, 1), "")'
    else:
        fact_ly = kwargs.get('fact_ly')
        fact_ty = kwargs.get('fact_ty')
        calculation_ly = f'=IF(SUM(OFFSET({fact_ly}{row},0,-($B$1-1)):{fact_ly}{row})=0,"",SUM(OFFSET({fact_ly}{row},0,-($B$1-1)):{fact_ly}{row}))'
        calculation_ty = f'=IF(SUM(OFFSET({fact_ty}{row},0,-($B$1-1)):{fact_ty}{row})=0,"",SUM(OFFSET({fact_ty}{row},0,-($B$1-1)):{fact_ty}{row}))'
    return calculation_ly, calculation_ty

In [338]:
def write_calculation(line_ws, name, row, start_column, letters, calculation_ly, calculation_ty, calculation_name, formulas_length):
    """
    Writes a calculation formula to the specified Excel worksheet.

    Args:
        line_ws (xlsxwriter.worksheet.Worksheet): The worksheet where the calculation will be written.
        name (str): The name of the calculation type (e.g., "Average Price per Unit", "Share in Units Sold").
        row (int): The row number where the calculation will be written.
        start_column (int): The starting column index for the calculation.
        letters (list): A list of Excel column letters corresponding to column indices.
        percent_format (xlsxwriter.format.Format): The format object for percentage formatting.
        calculation_ly (str): The formula for the "last year" calculation.
        calculation_ty (str): The formula for the "this year" calculation.
        calculation_name (str): The type of calculation (e.g., "dynamics", "change").
        formulas_length (int): The number of rows to apply conditional formatting.

    Raises:
        ValueError: If an unknown calculation name type is provided.

    Notes:
        - This function writes the calculation formulas for both "last year" and "this year" data.
        - It applies conditional formatting and writes the calculated difference or dynamics based on the calculation type.
        - The function supports different formatting types such as numeric, average, and percentage.
    """
    total_row = 3
    total_mw_row = 4
    
    line_ws.conditional_format(total_row - 1, start_column + 2, total_row - 1 + formulas_length, start_column + 2, {'type': 'icon_set',
                              'icon_style': '3_arrows'})

    if name == "Average Price per Unit":
        type = "average"
    elif name in ["Share in Units Sold", "Value Share", "Numeric Distribution TOTAL", "Weighted Value Distribution"]:
        type = "percent"
    else:
        type = "numeric"
    
    if row == total_row:
        format = concat_format(workbook, color="gray", type=type)
    elif row == total_mw_row:
        format = concat_format(workbook, color="light_gray", type=type)
    else:
        format = concat_format(workbook, color="", type=type)

    line_ws.write(row - 1, start_column, calculation_ly, format)
    line_ws.write(row - 1, start_column + 1, calculation_ty, format)
    

    if row == total_row:
        format = concat_format(workbook, color="gray", type="percent")
    elif row == total_mw_row:
        format = concat_format(workbook, color="light_gray", type="percent")
    else:
        format = concat_format(workbook, color="", type="percent")

    calculation_ly_column = letters[start_column]
    calculation_ty_column = letters[start_column + 1]
    if calculation_name == 'dynamics':
        line_ws.write(row - 1, start_column + 2, f'=IFERROR({calculation_ty_column}{row}/{calculation_ly_column}{row}-1, "")', format)
    elif calculation_name == 'change':
        line_ws.write(row - 1, start_column + 2, f'=IFERROR({calculation_ty_column}{row}-{calculation_ly_column}{row}, "")', format)
    else:
        raise ValueError(f"Unknown calculation name type: {calculation_name}")

In [339]:
start_row = 3
start_column = 2 + len(parameters)
prepared_facts = {}

In [340]:
for idx, name in enumerate(CALCULATION_NAMES):
    if name in facts_ready_to_prepare:
        column_num = facts_ready_to_prepare[name]
        prepared_fact = prepare_facts(column_num)
        prepared_facts[name] = prepared_fact
    
    calculation_name = CALCULATION_LEGEND[name]
    for row in range(start_row, FORMULAS_LENGTH):
        prepared_calculation = prepare_calculation(name, row, start_column, MERGE_RANGE, letters, fact_ly=prepared_fact[0], fact_ty=prepared_fact[1], prepared_facts=prepared_facts)
        
        write_calculation(
            line_ws, name, row, start_column, letters,
            calculation_ly=prepared_calculation[0], calculation_ty=prepared_calculation[1], calculation_name=calculation_name, formulas_length=FORMULAS_LENGTH
        )
        
    start_column += MERGE_RANGE

In [341]:
line_ws.freeze_panes(2, 2)

In [342]:
line_ws.autofit()

In [343]:
line_ws.set_row(1, 30)
line_ws.set_column(0, 1, 15)
line_ws.set_column(2, 2 + len(calculations) + len(CALCULATION_NAMES) * MERGE_RANGE, 12)
line_ws.set_column(2 + len(parameters), 2 + len(calculations) + len(CALCULATION_NAMES) * MERGE_RANGE, 10)
line_ws.set_column(2 + len(parameters) + MERGE_RANGE, 2 + len(parameters) + MERGE_RANGE + 1, 12)

0

In [344]:
writer.close()